In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import concatenate
import tensorflow_addons as tfa
from keras.layers import Flatten, Dense, Dropout
import tifffile

from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import os

import scipy
from numba import cuda 
from uszipcode import SearchEngine

2023-04-08 19:54:26.807558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 19:54:27.689986: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

In [2]:
# !pip install numpy tensorflow matplotlib scipy
# # !pip install pandas
# !pip install tifffile
# !pip install scikit-learn
# !pip install imagecodecs
# !pip install tensorflow-addons
# !pip install numba
# !pip install ipyplot
# !pip install uszipcode

In [3]:
def reset_gpu():
    device = cuda.get_current_device()
    device.reset()

In [4]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
   # plt.plot(history.history['accuracy'], label='accuracy')
   # plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.ylim([0, 
            max(
                max(history.history['loss']),
                max(history.history['val_loss'])
            )])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

In [5]:
# tiles = pd.read_csv(r'Data/Tiles_expanded.csv')
tiles_expanded_rv = pd.read_csv(r'Data/Tiles_binned.csv')#.drop(['Unnamed: 0'],axis=1)
# tiles_expanded_rv = pd.get_dummies(tiles_expanded_rv, columns = ['RTTYP'])

In [6]:
tiles_expanded_rv.head()

,Tile_ID,Long2,Lat2,Long1,Lat1,Mid_lat,Mid_long,Stop_Signs,Paving_historical,Paving_future,Bus_stop,Collisions_Future,Collisions_Historical,RTTYP_I,RTTYP_M,RTTYP_O,RTTYP_S,RTTYP_U,bin
0,36,-122.514446,37.779636,-122.513306,37.778732,37.779184,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0
1,37,-122.514446,37.778732,-122.513306,37.777829,37.778280,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0
2,151,-122.513306,37.779636,-122.512166,37.778732,37.779184,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0
3,152,-122.513306,37.778732,-122.512166,37.777829,37.778280,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0
4,153,-122.513306,37.777829,-122.512166,37.776925,37.777377,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0


In [8]:
tiles_expanded_rv['bin'].value_counts()

0    8290
1    1860
2     249
3      72
Name: bin, dtype: int64

In [7]:
# tiles_expanded_rv['Collisions_Future_binary']=tiles_expanded_rv['Collisions_Future'].apply(lambda x: 1 if x > 0 else 0)
# tiles_expanded_rv['Collisions_Historical_binary']=tiles_expanded_rv['Collisions_Historical'].apply(lambda x: 1 if x > 0 else 0)

In [17]:
tiles_expanded_rv.rename(columns={"bin": "bins_numeric"})

,Tile_ID,Long2,Lat2,Long1,Lat1,Mid_lat,Mid_long,Stop_Signs,Paving_historical,Paving_future,Bus_stop,Collisions_Future,Collisions_Historical,RTTYP_I,RTTYP_M,RTTYP_O,RTTYP_S,RTTYP_U,bins_numeric,zip_code
0,36,-122.514446,37.779636,-122.513306,37.778732,37.779184,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94121
1,37,-122.514446,37.778732,-122.513306,37.777829,37.778280,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94121
2,151,-122.513306,37.779636,-122.512166,37.778732,37.779184,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94121
3,152,-122.513306,37.778732,-122.512166,37.777829,37.778280,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94121
4,153,-122.513306,37.777829,-122.512166,37.776925,37.777377,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10466,15733,-122.359390,37.728130,-122.358250,37.727227,37.727679,-122.358820,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94124
10467,15734,-122.359390,37.727227,-122.358250,37.726323,37.726775,-122.358820,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94124
10468,15735,-122.359390,37.726323,-122.358250,37.725420,37.725871,-122.358820,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94124
10469,15848,-122.358250,37.728130,-122.357110,37.727227,37.727679,-122.357680,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,94124


In [22]:
# tiles_expanded_rv['Long2']==tiles_expanded_rv['Long1']

0        False
1        False
2        False
3        False
4        False
         ...  
10466    False
10467    False
10468    False
10469    False
10470    False
Length: 10471, dtype: bool

In [9]:
# # Search zipcode within 30 miles, ordered from closest to farthest
search = SearchEngine()
def get_zipcode(coordinates,radius):
    result = search.by_coordinates(coordinates[0], coordinates[1], radius=radius,returns=1)
    if len(result)==0:
        return get_zipcode(coordinates,radius*2)
    else:
        return result[0]
# result # by default 5 results returned

In [10]:
coords_list = list(zip(tiles_expanded_rv.Mid_lat, tiles_expanded_rv.Mid_long))
zips_list = []
for x in coords_list:
    zips_list.append(get_zipcode(x,0.031).zipcode)

In [11]:
tiles_expanded_rv['zip_code']=zips_list

In [21]:
tiles_expanded_rv = tiles_expanded_rv.rename(columns={"bin": "bins_numeric"})

In [24]:
pd.get_dummies(tiles_expanded_rv['zip_code'])

,94101,94102,94104,94105,94107,94108,94109,94110,94111,94112,...,94129,94130,94131,94132,94133,94134,94141,94143,94158,94188
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10466,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10467,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
one_hot_zc = pd.concat(
    [
        tiles_expanded_rv,
        pd.get_dummies(tiles_expanded_rv['zip_code'])
    ], axis=1
)

In [28]:
one_hot_zc.columns

Index(['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical', 'RTTYP_I', 'RTTYP_M',
       'RTTYP_O', 'RTTYP_S', 'RTTYP_U', 'bins_numeric', 'zip_code', '94101',
       '94102', '94104', '94105', '94107', '94108', '94109', '94110', '94111',
       '94112', '94114', '94115', '94116', '94117', '94118', '94121', '94122',
       '94123', '94124', '94127', '94129', '94130', '94131', '94132', '94133',
       '94134', '94141', '94143', '94158', '94188'],
      dtype='object')

In [29]:
from pathlib import Path
file_path = Path('/home/ubuntu/210_Capstone_Aditya_Arisa_Noriel/Data/Tiles_binned_zipcode_rebinned.csv')
one_hot_zc.to_csv(file_path,index=False)